In [1]:
import gymnasium as gym
from ptan.experience import ExperienceFirstLast, ExperienceSourceFirstLast
import ptan
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import typing as tt


HIDDEN_SIZE = 128
BATCH_SIZE = 16
TGT_NET_SYNC = 10
GAMMA = 0.9
REPLAY_SIZE = 1000
LR = 1e-3
EPS_DECAY = 0.99


class Net(nn.Module):
    def __init__(self, obs_size: int, hidden_size: int, n_actions: int):
        super(Net, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(obs_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, n_actions)
        )

    def forward(self, x):
        return self.net(x.float())


@torch.no_grad()
def unpack_batch(batch: tt.List[ExperienceFirstLast], net: Net, gamma: float):
    states = []
    actions = []
    rewards = []
    done_masks = []
    last_states = []
    for exp in batch:
        states.append(exp.state)
        actions.append(exp.action)
        rewards.append(exp.reward)
        done_masks.append(exp.last_state is None)
        if exp.last_state is None:
            last_states.append(exp.state)
        else:
            last_states.append(exp.last_state)

    states_v = torch.as_tensor(np.stack(states))
    actions_v = torch.tensor(actions)
    rewards_v = torch.tensor(rewards)
    last_states_v = torch.as_tensor(np.stack(last_states))
    last_state_q_v = net(last_states_v)
    best_last_q_v = torch.max(last_state_q_v, dim=1)[0]
    best_last_q_v[done_masks] = 0.0
    return states_v, actions_v, best_last_q_v * gamma + rewards_v


if __name__ == "__main__":
    env = gym.make("CartPole-v1")
    obs_size = env.observation_space.shape[0]
    n_actions = env.action_space.n

    net = Net(obs_size, HIDDEN_SIZE, n_actions)
    tgt_net = ptan.agent.TargetNet(net)
    selector = ptan.actions.ArgmaxActionSelector()
    selector = ptan.actions.EpsilonGreedyActionSelector(epsilon=1, selector=selector)
    agent = ptan.agent.DQNAgent(net, selector)
    exp_source = ptan.experience.ExperienceSourceFirstLast(env, agent, gamma=GAMMA)
    buffer = ptan.experience.ExperienceReplayBuffer(exp_source, buffer_size=REPLAY_SIZE)
    optimizer = optim.Adam(net.parameters(), LR)

    step = 0
    episode = 0
    solved = False

    while True:
        step += 1
        buffer.populate(1)

        for reward, steps in exp_source.pop_rewards_steps():
            episode += 1
            print(f"{step}: episode {episode} done, reward={reward:.2f}, "
                  f"epsilon={selector.epsilon:.2f}")
            solved = reward > 150
        if solved:
            print("Whee!")
            break
        if len(buffer) < 2*BATCH_SIZE:
            continue
        batch = buffer.sample(BATCH_SIZE)
        states_v, actions_v, tgt_q_v = unpack_batch(batch, tgt_net.target_model, GAMMA)
        optimizer.zero_grad()
        q_v = net(states_v)
        q_v = q_v.gather(1, actions_v.unsqueeze(-1)).squeeze(-1)
        loss_v = F.mse_loss(q_v, tgt_q_v)
        loss_v.backward()
        optimizer.step()
        selector.epsilon *= EPS_DECAY

        if step % TGT_NET_SYNC == 0:
            tgt_net.sync()

Gym has been unmaintained since 2022 and does not support NumPy 2.0 amongst other critical functionality.
Please upgrade to Gymnasium, the maintained drop-in replacement of Gym, or contact the authors of your software and request that they upgrade.
Users of this version of Gym should be able to simply replace 'import gym' with 'import gymnasium as gym' in the vast majority of cases.
See the migration guide at https://gymnasium.farama.org/introduction/migration_guide/ for additional information.


24: episode 1 done, reward=23.00, epsilon=1.00
39: episode 2 done, reward=15.00, epsilon=0.93
64: episode 3 done, reward=25.00, epsilon=0.72
74: episode 4 done, reward=10.00, epsilon=0.66
93: episode 5 done, reward=19.00, epsilon=0.54
112: episode 6 done, reward=19.00, epsilon=0.45
134: episode 7 done, reward=22.00, epsilon=0.36
144: episode 8 done, reward=10.00, epsilon=0.32
157: episode 9 done, reward=13.00, epsilon=0.28
171: episode 10 done, reward=14.00, epsilon=0.25
181: episode 11 done, reward=10.00, epsilon=0.22
194: episode 12 done, reward=13.00, epsilon=0.20
203: episode 13 done, reward=9.00, epsilon=0.18
213: episode 14 done, reward=10.00, epsilon=0.16
224: episode 15 done, reward=11.00, epsilon=0.15
234: episode 16 done, reward=10.00, epsilon=0.13
243: episode 17 done, reward=9.00, epsilon=0.12
251: episode 18 done, reward=8.00, epsilon=0.11
260: episode 19 done, reward=9.00, epsilon=0.10
270: episode 20 done, reward=10.00, epsilon=0.09
281: episode 21 done, reward=11.00, ep